In [23]:
from polyname import check_database
from polyname import remove_duplicates
from polyname import search_names
from polyname import polymer_extract
from polyname import strip_poly
from polyname import lowercase
from polyname import get_pubchem_smiles
from polyname import get_cirpy_smiles

import pandas as pd


In [18]:
#POLYMER EXTRACTION STEP
#Input : chemical names outputted by the extraction step
all_names_path = "example/all_names.txt"
#isolate the polymers
polymer_list = polymer_extract(all_names_path)
#print DF
#convert it to dataframe and write to a textfile with all isolated polymer names
data = pd.DataFrame (polymer_list, columns = ['Isolated_poly_names'])
print(data)
data.to_csv("example/polymername_final.txt", index = False, header = False)

                 Isolated_poly_names
0                      Poly(Llysine)
1                 Poly(pentapeptide)
2                   Poly(Lglutamate)
3            Poly(\u03b1-amino acid)
4            Poly(\u03b1-amino acid)
..                               ...
76                      Poly(C-Cu11)
77                      Poly(C-CuII)
78              Poly(cytidylic acid)
79  Poly(EGDMA-HEMA Alkali Blue 613)
80                Poly(vinylalcohol)

[81 rows x 1 columns]


In [19]:
#CHECK DATABASE STEP
#Input : database file name and polymername_final.txt
database_path = "example/polyname_smiles_test.csv"
all_polymers = "example/polymername_final.txt"
#check database for each polymer name and remove duplicates
name,smile = check_database(all_polymers,database_path)
final_name,final_smile = remove_duplicates(name,smile)
#return a list with polymer names not found in database, Input: the all polymers names and final names found in DB
search_name = search_names(all_polymers,final_name)
print(search_name)

#add polymer names and monomer smiles 
df = pd.DataFrame(final_name, columns = ["Polymer Names"])
df.insert(1,'Monomer Smiles', final_smile)
print(df)
df.to_csv('example/database_extract.csv', index = None)
#write the names not found in DB into a df2
lowercase_poly = monomer.lowercase(search_name)
df2 = pd.DataFrame(lowercase_poly, columns = ["polymername"])
print(df2)
df2.to_csv('example/polymer_extract_still.txt', index = None)

['Poly(Llysine)', 'Poly(pentapeptide)', 'Poly(Lglutamate)', 'Poly(\\u03b1-amino acid)', 'Poly(\\u03b1-amino acid)', 'Poly(L-glutamic acid)', 'Poly(a-amino acid)', 'Poly(meric primary amine)', 'Poly(styrene)', 'Poly(unsaturated cycloolefine)', 'Poly(styrene)', 'Poly(styrene)', 'Poly(butadiene)', 'Poly(isobutylene-co-diene)', 'Poly(methacrylate-co-butadiene)', 'Poly(butyl methacrylate-co-butadiene)', 'Poly(styrene)', 'Poly(butyl methacrylate)', 'Poly(Llysine)', 'Poly(pentapeptide)', 'Poly(Lglutamate)', 'Poly(\\u03b1-amino acid)', 'Poly(\\u03b1-amino acid)', 'Poly(L-glutamic acid)', 'Poly(a-amino acid)', 'Poly(meric primary amine)', 'Poly(styrene)', 'Poly(unsaturated cycloolefine)', 'Poly(styrene)', 'Poly(styrene)', 'Poly( 4000)', 'Poly(l-glutamate)', 'Poly(styrene)', 'Poly(ether)', 'Poly(phenylene)', 'Poly(styrene)', 'Poly(phenylene)', 'Poly(ester)', 'Poly(ester)', 'Poly( PIN-MAMnII)', 'Poly(cytidylic acid sodium)', 'Poly(uridylic acid)', 'Poly(inosinic acid [6])', 'Poly(CCuII)', 'Poly(c

In [32]:
#EXTRACT MONOMER SMILES come back to this
#Extract monomer smiles from the polymers that still need to be extracted 
polymers_to_extract = 'example/polymer_extract_still.csv'
#read textfile into dataframe
data2 = pd.read_table(polymers_to_extract)
print(data2)
#get unique polymer names 
dfnames2 = pd.DataFrame(data2.polymername.unique(),columns=['polyname'])
print(dfnames2)
stripstrings = [['poly(',')'],['poly[(',')]'],['poly[',']'],['poly{','}'],['poly-','']]
dfnames2['polyname_1'] = dfnames2.apply(lambda row: monomer.strip_poly(row,stripstrings),axis=1)
print(dfnames2)
#pubchem is busy
monomer_names2 = dfnames2.apply(lambda row: monomer.strip_poly(row,stripstrings),axis=1)
dfnames2['Pubchem Smiles'] = monomer_names2.apply(lambda row: monomer.get_pubchem_smiles(row))
dfnames2['Cirpy Smiles'] = monomer_names2.apply(lambda row: get_cirpy_smiles(row))
print(dfnames2)
dfnames2.to_csv("example_extract_monomers.csv", index = False)

                                          polymername
0                                       poly(llysine)
1                                  poly(pentapeptide)
2                                    poly(lglutamate)
3                             poly(\u03b1-amino acid)
4                               poly(l-glutamic acid)
5                                  poly(a-amino acid)
6                           poly(meric primary amine)
7                                       poly(styrene)
8                      poly(unsaturated cycloolefine)
9                                     poly(butadiene)
10                         poly(isobutylene-co-diene)
11                    poly(methacrylate-co-butadiene)
12              poly(butyl methacrylate-co-butadiene)
13                           poly(butyl methacrylate)
14                                        poly( 4000)
15                                  poly(l-glutamate)
16                                        poly(ether)
17                          

                                             polyname  \
0                                       poly(llysine)   
1                                  poly(pentapeptide)   
2                                    poly(lglutamate)   
3                             poly(\u03b1-amino acid)   
4                               poly(l-glutamic acid)   
5                                  poly(a-amino acid)   
6                           poly(meric primary amine)   
7                                       poly(styrene)   
8                      poly(unsaturated cycloolefine)   
9                                     poly(butadiene)   
10                         poly(isobutylene-co-diene)   
11                    poly(methacrylate-co-butadiene)   
12              poly(butyl methacrylate-co-butadiene)   
13                           poly(butyl methacrylate)   
14                                        poly( 4000)   
15                                  poly(l-glutamate)   
16                             